<a href="https://colab.research.google.com/github/olaviinha/inhagcutils/blob/master/inhagcutils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# inhagcutils

A set of modules and functions I seem to need in too many Colab Notebooks.

### Import modules

In [ ]:
import sys, os, ntpath, string, random, librosa, librosa.display, IPython, shutil, math, psutil, datetime, requests, pytz
from glob import glob
from os.path import isdir, join
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from distutils.dir_util import copy_tree
from urllib.request import urlopen
from PIL import Image
from google.colab import drive, output, files
from base64 import b64encode
import subprocess

### Common variables

In [ ]:
# Root dirs
drive_root = "/content/drive/MyDrive/"
dir_tmp = "/content/tmp/"

# Silence stuff
shell_q = "> /dev/null 2>&1"
ffmpeg_q = "-hide_banner -loglevel panic" # -hide_banner -loglevel panic
sox_q = "-q" # -S, -q
youtube_q = "-q" # -q
wget_q = "-q" # -q
git_q = "-q" # -q
curl_q = "-s" # -s
pip_q = "-q" # -q

# Plot colorspace
plot_bg = '#272822'
plot_wav = '#d5d5d5'

# Audio formats for FFmpeg
wav_16 = "-c:a pcm_s16le -ar 16000 -ac 2 " # Signed 16-bit 44.1kHz Stereo
wav_44 = "-c:a pcm_s16le -ar 44100 -ac 2 " # Signed 16-bit 44.1kHz Stereo
wav_48 = "-c:a pcm_s24le -ar 48000 -ac 2"  # Signed 24-bit 48kHz Stereo
mp3_192 = "-vn -ar 44100 -ac 2 -b:a 192k"  # 44.1kHz 192kbps Stereo
mp3_320 = "-vn -ar 48000 -ac 2 -b:a 320k" # 48kHz 320kbps Stereo

# Packed files
zip_extensions = ('zip', 'gz')

# Help
help = []

### Functions: general

In [ ]:
help.append({'general': [
  'op(typex, msg, value=\'\', time=False)',
  'download_raw(url, destination)',
  'end_session()',
  'timestamp(no_slash=False, human_readable=False, helsinki_time=True, date_only=False)',
  'rnd_str(length)',
  'gen_id(type=\'short\') # short, long, timestamp',
  'pick_random(list, nr=1, mode=\'sentence\') # sentence, list',
  'what_gpu()',
  'is_hosted_runtime()',
  'fix_path(path, add_slash=False)',
  'path_ext(path, only_ext=False)',
  'basename(path)',
  'check_input_type(path)',
  'most_frequent(list)',
  'slug(path)',
  'concat_list(p, s)',
  'odds(odds)',
  'install_conda(packages)',
  'is_zip(path)',
  'is_zipzip(path)',
  'is_gz(path)',
  'sys_mem()',
  'gpu_mem()',
  'get_from_txt(path, info, display_not_found=False)',
  'append_txt(path, content)',
  'dl_btn(file_path)',
  'replicate(list, n_times)',
  'helpme(section)'
]})

# Print colors
class c:
  title = '\033[96m'
  ok = '\033[92m'
  okb = '\033[94m'
  warn = '\033[93m'
  fail = '\033[31m'
  endc = '\033[0m'
  bold = '\033[1m'
  dark = '\33[90m'
  u = '\033[4m'

# Input (class, str, str): message type, colored message, plain message
# Output: colored text
def op(typex, msg, value='', time=False):
  if time == True:
    stamp = timestamp(human_readable=True)
    typex = c.dark+stamp+' '+typex
  if value != '':
    print(typex+msg+c.endc, end=' ')
    print(value)
  else:
    print(typex+msg+c.endc)

# Input (str): file or directory path
# Output: full drive paths of given dir or all files in given dir
def handle_path(input, arr=True):
  global mount_drive, drive_root
  if os.path.isfile(drive_root+input):
    inputs = [drive_root+input]
    dir_in = path_dir(drive_root+input)
  elif input != '' and os.path.isdir(drive_root+input):
    dir_in = drive_root+fix_path(input)
    inputs = glob(dir_in+'/*')
  elif os.path.isdir(drive_root+input) and '*' in input:
    dir_in = path_dir(drive_root+input)
    inputs = glob(drive_root+input)
  elif mount_drive == False:
    %cd {dir_tmp_in}
    uploaded = files.upload()
    inputs = glob(dir_tmp_in+'/*')
  if len(inputs):
    if arr == True:
      return inputs
    else:
      return inputs[0]
  else:
    op(c.fail, 'FAIL!', 'Input should be a path to a file or a directory.')
    sys.exit('Input not understood.')
    return None


# Input (str): URL to download
# Output (str): Downloaded local file
def download_raw(url, destination):
  if 'github.com' in url:
    url = url.replace('github.com', 'raw.githubusercontent.com').replace('blob/', '').replace('tree/', '')
  if 'dropbox.com' in url:
    url = url.replace('www.dropbox.com', 'dl.dropboxusercontent.com').replace('dl=0', 'raw=1').replace('dl=1', 'raw=1');
  filename = path_leaf(url)
  if os.path.isdir(destination):
    destination = fix_path(destination)+filename
  print('wget', url, destination)
  !wget {url} -O {destination}
  return destination

# Disconnect from runtime
def end_session():
  from google.colab import runtime
  op(c.title, 'Intentional runtime termination.', time=True)
  runtime.unassign()

# Input (boolean, boolean, boolean):  no separators, separators + space between date and time, convert to Helsinki time
# Output (str): timestamp YYYYMMDDHHIIss or YYYY-MM-DD_HHIIss or YYYY-MM-DD HH:II:ss
def timestamp(no_slash=False, human_readable=False, helsinki_time=True, date_only=False):
  if helsinki_time == True:
    dt = datetime.datetime.now(pytz.timezone('Europe/Helsinki'))
  else:
    dt = datetime.datetime.now()
  if no_slash == True:
    dt = dt.strftime("%Y%m%d%H%M%S")
  else:
    if human_readable == True:
      dt = dt.strftime("%Y-%m-%d %H:%M:%S")
    else:
      if date_only == True:
        dt = dt.strftime("%Y-%m-%d")
      else:
        dt = dt.strftime("%Y-%m-%d_%H%M%S")
  return dt;

# Input (int): number
# Output (str): random string of <number> characters long
def rnd_str(length):
  letters = string.ascii_lowercase
  result_str = ''.join(random.choice(letters) for i in range(length))
  return result_str

# Input (str): type 'short', 'long', 'timestamp'
# Output (str): Human readable 6 char ID key or timestamp
def gen_id(type='short'):
  id = ''
  if type == 'timestamp':
    id = timestamp()
  if type == 'short':
    id = requests.get('https://api.inha.asia/k/?type=short').text
  if type == 'long':
    id = requests.get('https://api.inha.asia/k').text
  return id

def pick_random(list, nr=1, mode='sentence'):
  vals = random.sample(list, nr)
  if mode == 'sentence':
    vals = ', '.join(vals)
    vals = vals[::-1].replace(', '[::-1], ' and '[::-1], 1)[::-1]
  elif mode == 'list':
    vals = ', '.join(vals)
  return vals

# Input: -
# Output: Name and type of used GPU
def what_gpu():
  x = !nvidia-smi
  g = ''.join(x);
  restart = False
  gpu = 'None'
  if 'A100' in g:
    gpu = 'A100'
    restart = True
  elif 'V100' in g:
    gpu = 'V100'
  elif 'P100' in g:
    gpu = 'P100'
  elif 'T4' in g:
    gpu = 'T4'
  else:
    gpu = x
  return gpu
whatGPU = what_gpu

# Input: -
# Output (boolean): ipynb is running on _hosted_ Colab runtime
def is_hosted_runtime():
  return 'google.colab' in sys.modules

# Input (str, boolean): path, remove first slash
# Output (string): path with missing / at the end, opt: remove from beginning
def fix_path(path, add_slash=False):
  if path.endswith('/'):
    path = path #path[:-1]
  if not path.endswith('/'):
    path = path+"/"
  if path.startswith('/') and add_slash == True:
    path = path[1:]
  return path

# Input (str): path
# Output (str): filename with extension
def path_leaf(path):
  head, tail = ntpath.split(path)
  return tail or ntpath.basename(head)

# Input (str): file path
# Output (str): enclosing directory
def path_dir(path):
  return path.replace(path_leaf(path), '')

# Input (str, boolean): file path, keep dot
# Output (str): file extension with or without the .dot
def path_ext(path, only_ext=False):
  filename, extension = os.path.splitext(path)
  if only_ext == True:
    extension = extension[1:]
  return extension

# Input (str): path
# Output (str): filename without extension
def basename(path):
  filename = os.path.basename(path).strip()#.replace(" ", "_")
  filebase = os.path.splitext(filename)[0]
  return filebase

# Input (str): path
# Output (str): "dir", "file", "youtube", "link" or "unknown"
def check_input_type(path):
  if os.path.isdir(path):
    input_type = "dir"
    input = fix_path(path)
  elif os.path.isfile(path):
    input_type = "file"
  elif "://" in path and "youtu" in path:
    input_type = "youtube"
  elif "://" in path:
    input_type = "link"
  else:
    input_type = "unknown"
  return input_type

# Input (list): list
# Output (any): modal value of list
def most_frequent(list):
  freq = max(set(list), key = list.count)
  print(str(list.count(freq))+' out of '+str(len(list)), 'items have a value of', str(freq))
  return freq

# Input (str): stupid filename, e.g. "any long% weird !filename (like this).wav"
# Output (str): cool filename, e.g. "any_long_weird_filename_like_this.wav"
def slug(s):
  valid_chars = "-_. %s%s" % (string.ascii_letters, string.digits)
  file = ''.join(c for c in s if c in valid_chars)
  file = file.replace(' ','_')
  return file

# Input (list): list
# Output (str): every -param item -param in -param list
def concat_list(p, s):
  p=' '+p+' '
  return (p+p.join(s))

# Input (float): probability of True
# Output (boolean): True on given probability, otherwise False
def odds(probability):
  return random.random() < probability

# Input: conda packages
# Output: -
def install_conda(packages):
  !wget {wget_q} -c https://repo.continuum.io/archive/Anaconda3-5.1.0-Linux-x86_64.sh
  !chmod +x Anaconda3-5.1.0-Linux-x86_64.sh
  !bash ./Anaconda3-5.1.0-Linux-x86_64.sh -b -f -p /usr/local
  import sys
  sys.path.append('/usr/local/lib/python3.6/site-packages/')
  !conda install -q -y {packages}

# Input (str): path
# Output (boolean): True if file is packed
def is_zip(path):
  return path.lower().endswith(zip_extensions)

def is_zipzip(path):
  return path_ext(path).lower() == '.zip'

def is_gz(path):
  return path_ext(path).lower() == '.gz'

def sys_mem():
  x = !grep MemTotal /proc/meminfo
  return x
sysMem = sys_mem

def gpu_mem():
  x = !nvidia-smi --query-gpu=memory.total --format=csv
  return x
gpuMem = gpu_mem

# Input (str): Path to txt-file, info to fetch (e.g. "Seed:")
# Output (str): Value, trimmed after ":""
def get_from_txt(txt_file, info, display_not_found=False):
  import os
  res = '<not found>' if display_not_found == True else ''
  if os.path.isfile(txt_file):
    txt = open(txt_file, 'r')
    lines = txt.readlines()
    for line in lines:
      if info.lower() in line.lower():
        res = line.split(':')[1].strip()
  return res

# Input (str, str): Path to txt-file, content to append-save
# Output: -
def append_txt(txt_file, content):
  txt = open(txt_file, 'a+')
  txt.writelines(content+'\n')
  txt.close();


def downgrade_python(v=3.8):
  !apt-get update -y
  !apt-get install python{v}
  !update-alternatives --set python3 /usr/bin/python{v}
  print()
  !python --version
  print()
  !curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
  !python get-pip.py
  print()

def replicate(arr, times):
  return [val for val in arr for _ in range(times)]

# Input (str): general, image or audio
# Output: inhagcutils cheatsheet
def helpme(section=None):
  if section != None:
    for i, h in enumerate(help):
      if section in h:
        op(c.title, section.capitalize()+':')
        for x in h[section]:
          print('-', x )
        print()
  else:
    for h in help:
      for k, v in h.items():
        op(c.title, k.capitalize()+':')
        for cc in v:
          print('-', cc)
        print()

# Input (str): space separated numeric values and number of steps
# Output (list): Given values interpolated to given number of steps
def stepify(values, steps):
  value_states = [float(v) for v in values.split(' ')]
  value_states_total = len(value_states)
  value_steps = []
  gaps = value_states_total-1
  for i in range(gaps):
    start = value_states[i]
    end = value_states[i+1]
    f = np.linspace(start, end, round(steps/gaps))
    value_steps.extend(f)
  return value_steps

### Functions: image related

In [ ]:
help.append({'image': [
  'list_images(path, exclude_pattern=[], format=\'all\')',
  'aspect_ratio_to_pixels(aspect_ratio=\'1:1\', max_px=704*704)',
  'cover_crop_resize(img=PILimg, new_width, new_height, from_top=False)',
  'google_image_search(api_key, q, nr=0, size=\'huge\', optimize=True)',
  'fetch(url, save_as)',
  'get_prompt(filename, and_seed=False)',
  'png_or_jpg(path)'
]})

# Input: directory path (str), exclude pattern (str), format (str)
# Output (list): list of audiofiles in dir
from glob import glob
from os.path import isdir, join
def list_images(path, exclude_pattern=[], format='all'):
  formats = ['jpg', 'jpeg', 'png', 'gif', 'webp']
  if format != 'all':
    formats = [format]
  imagefiles = []
  formats_capital = [f.capitalize() for f in formats]
  formats_upper = [f.upper() for f in formats]
  formats.extend(formats_capital)
  formats.extend(formats_upper)
  formats = ['*.'+f for f in formats]
  for ext in formats:
    imagefiles.extend(glob(join(path, ext)))

  if len(exclude_pattern) > 0:
    filtered_imagefiles = []
    for f in imagefiles:
      if not any(e in f for e in exclude_pattern): filtered_imagefiles.append(f)
    imagefiles = filtered_imagefiles
  imagefiles.sort()
  return imagefiles

def aspect_ratio_to_pixels(aspect_ratio='1:1', max_px=704*704):
  w, h = [int(x) for x in aspect_ratio.split(':')]
  nw = math.floor(math.sqrt(max_px/h*w));
  nh = math.floor(math.sqrt(max_px/w*h));
  nw = (nw+0x20)&(~0x3f);
  nh = (nh+0x20)&(~0x3f);
  return nw, nh
aspectRatioToPixels = aspect_ratio_to_pixels

# Input (PIL image, int, int): image, new width, new height
# Output: PIL image cropped and resized
def cover_crop_resize(img, new_width, new_height, from_top=False):
  width, height = img.size
  if height < new_height:
    height_percent = (new_height / float(height))
    width_size = int((float(width) * float(height_percent)))
    img = img.resize((width_size, new_height), Image.NEAREST)
  width, height = img.size
  if width < new_width:
    width_percent = (new_width / float(width))
    height_size = int((float(height) * float(width_percent)))
    img = img.resize((new_width, height_size), Image.NEAREST)
  width, height = img.size
  nwp = new_width/new_height
  nhp = new_height/new_width
  left = 0
  top = 0
  right = width
  bottom = height
  nh = nhp * width
  test_top =  int((height-nh)/2)
  if test_top > 0:
    # crop from bottom only (e.g. portraits)
    if from_top == True:
      top = int((height-nh)/4)
      bottom = int((height/8+nh))
    # crop from top & bottom
    else:
      top = int((height-nh)/2)
      bottom = int((height+nh)/2)
  else:
    # crop left & right
    nw = nwp * height
    left = int((width-nw)/2)
    right = int((width+nw)/2)
  cropped = img.crop((left, top, right, bottom))
  # Resize cropped image
  new_size = (new_width, new_height)
  resized = cropped.resize(new_size)
  return resized
coverCropResize = cover_crop_resize

# Input (str, str, int, str): Google API key, search query, number to return, image size
# Output: returns selected image URL from Google Image Search
def google_image_search(api_key, q, nr=0, size='huge', optimize=True):
  cx = '696a4696eb66d64ca'
  if optimize == True:
    q = q + ' -quote -quotes -screenshot -capture -logo -banner -stock -slide -shutterfly -getty -123rf -alamy -bigstock -shutterstock -dreamstime -canstock -colourbox -guardian'
  URL = 'https://www.googleapis.com/customsearch/v1?key='+api_key+'&cx='+cx+'&q='+q
  PARAMS = {
      'searchType': 'image',
      'imgSize': size,
      'fileType': 'jpg'
  }
  r = requests.get(url = URL, params = PARAMS)
  data = r.json()
  if 'items' in data:
    while nr not in data['items']:
      nr = nr-1
    response = data['items'][nr]['link']
  else:
    # Fallback
    response = 'https://picsum.photos/900/600'
  return response
googleImageSearch = google_image_search

# Fetch image (or other file) from URL
def fetch(url, save_as):
  headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}
  try:
    r = requests.get(url, stream=True, headers=headers, timeout=5)
    if r.status_code == 200:
      with open(save_as, 'wb') as f:
        r.raw.decode_content = True
        shutil.copyfileobj(r.raw, f)
      resp = r.status_code
    else:
      resp = 0
  except requests.exceptions.ConnectionError as e:
    r = 0
    resp = r
  return resp

# Get prompt from filename
def get_prompt(filename, and_seed=False):
  test = filename.replace('-', '')
  if not '_' in filename[:10] and (test[:10].isnumeric() and '-' in filename[:11]):
    prefix = ''
    initial_prompt = filename.split('-')[2:]
  else:
    prefix = ' '.join(filename.split('_')[:1])
    initial_prompt = filename.split('_')[2:]
  if len(initial_prompt) > 1:
    test2 = initial_prompt[-1]
  else:
    test2 = initial_prompt[0]
  if test2.replace('.png', '').isnumeric():
    if and_seed == True:
      initial_prompt[-1] = '\n\n'+initial_prompt[-1].replace('.png', '')
    else:
      initial_prompt[-1] = ''
  return ' '.join(initial_prompt)
getPrompt = get_prompt

# Return either png or jpeg path
def png_or_jpg(path):
  if os.path.isfile(path):
    return path
  else:
    if os.path.isfile(path.replace('.png', '.jpg')):
      return path.replace('.png', '.jpg')
    if os.path.isfile(path.replace('.jpg', '.png')):
      return path.replace('.jpg', '.png')
    if os.path.isfile(path+'.jpg'):
      return path+'.jpg'
    if os.path.isfile(path+'.png'):
      return path+'.png'
  return False
pngOrJpeg = png_or_jpg

### Interactive UI-elements for cell outputs


In [ ]:
def applyInhaCSS():
  display(IPython.display.HTML('''
    <style type="text/css">
      :root {
        --bg-color: #eee;
        --fg-color: #333;
        --radius: 4px;
        --border: 0;
        --bold: 600;
        --dimmed: #ccc;
        --darkened: #aaa;
        --v-margin: 10px;
      }
      input.inhads,
      button.inhads {
        display: inline-block;
        border: var(--border);
        border-radius: var(--radius);
        background: var(--bg-color);
        color: var(--fg-color);
        margin-top: var(--v-margin);
        margin-bottom: calc(var(--v-margin) * 2);
      }
      input.inhads {
        padding: 10px 15px;
        min-width: 30%;
      }
      input.inhads.input-with-button {
        border-top-left-radius: var(--radius);
        border-bottom-left-radius: var(--radius);
        border-top-right-radius: 0;
        border-bottom-right-radius: 0;
      }
      button.inhads {
        cursor: pointer;
      }
      button.inhads.input-button {
        display: inline-block;
        margin: 0;
        padding: 4.5px 10px;
        position: relative;
        top: 3px;
        font-size: 20px;
        border-top-left-radius: 0;
        border-bottom-left-radius: 0;
        border-top-right-radius: var(--radius);
        border-bottom-right-radius: var(--radius)
      }
      button.inhads.download-button {
        padding: 10px 15px;
        font-weight: var(--bold);
      }
      button.inhads.download-button.done {
        background: var(--dimmed);
      }
      button.inhads.download-button.done::before {
        content: '✔ ';
        color: #080;
      }
      .inhads.input-note {
        padding-left: 10px;
        display: none;
        font-weight: var(--bold);
      }
      .inhads.disabled {
        color: #666;
        background: var(--darkened);
      }
    </style>
  '''))

In [ ]:
# Input (str): File path to file located in Runtime or Drive
# Output: Download-button for given file
def dl_btn(file_path, show_path=False, show_filename=False):
  applyInhaCSS()
  id = rnd_str(8)
  view_path = file_path if show_path else path_leaf(file_path) if show_filename else ''
  def download_file():
    files.download(file_path)
  display(IPython.display.HTML('''
    <button class="inhads download-button" id="btn_'''+id+'''">Download</button> '''+view_path+'''
    <script>
      document.querySelector("#btn_'''+id+'''").onclick = () => {
        let btn = document.querySelector("#btn_'''+id+'''");
        google.colab.kernel.invokeFunction("notebook.download'''+id+'''", [], {});
        btn.innerHTML='Downloading...';
        btn.classList.add('disabled');
        btn.disabled=true;
        setTimeout(() => {
          btn.innerHTML='Downloaded';
          btn.classList.remove('disabled');
          btn.classList.add('done');
          btn.disabled=false;
        }, 2000);
      };
    </script>
  '''))
  output.register_callback('notebook.download'+id, download_file)

# Input (str): Free text
# Output: Input field with accompanying "Copy to clipboard"-button
def copyable(value):
  applyInhaCSS()
  id = rnd_str(8)
  display(IPython.display.HTML('''
    <input class="inhads input-with-button" id="field_'''+id+'''" type="text" value="'''+value+'''"><button class="inhads input-button" id="btn_'''+id+'''" title="Copy value" alt="Copy value">📋</button>
    <span class="inhads input-note" id="note_'''+id+'''">Copied!</span>
    <script>
      document.querySelector("#btn_'''+id+'''").onclick = () => {
        var copyText = document.getElementById("field_'''+id+'''");
        copyText.select();
        copyText.setSelectionRange(0, 99999);
        navigator.clipboard.writeText(copyText.value);
        document.getElementById("field_'''+id+'''").blur();
        document.querySelector("#note_'''+id+'''").style.display='inline';
        setTimeout(() => {
          document.querySelector("#note_'''+id+'''").style.display='none';
        }, 2000);
      };
    </script>
  '''))

In [ ]:
# Non-interactive
def show_vid(file_path, grayscale=False, width=640):
  mp4 = open(file_path,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  gs = "filter:gaeyscale(1);" if grayscale==True else ''
  IPython.display.display(IPython.display.HTML('''
  <video style="'''+gs+'''" width="'''+str(width)+'''" controls>
    <source src="%s" type="video/mp4">
  </video>
  ''' % data_url))

def show_im(img):
  IPython.display.display(IPython.display.Image(img))

### Functions: temp file handlers (deprecated)

In [ ]:
# Input (list): list of directory paths
# Output: -
def create_dirs(paths):
  for path in paths:
    if not os.path.isdir(path) and path != '':
      !mkdir "{path}"

# Input (list): list of directory paths
# Output: -
def remove_dirs(paths):
  for path in paths:
    if os.path.isdir(path):
      !rm -r "{path}"

# Input (list): list of directory paths
# Output: -
def reset_dirs(paths):
  remove_dirs(paths)
  create_dirs(paths)

# Input (list): list of directory path
# Output: -
def clean_dirs(paths):
  for path in paths:
    path = fix_path(path)
    dir = os.listdir(path)
    if len(dir) != 0:
      !rm {path}*

# Input (str): source directory path, destination directory path
# Output: -
def copy_all(source_dir, destination_dir):
  for f in glob(source_dir+'*'):
    shutil.copy(f, destination_dir)


### Functions: audio-related

In [ ]:
help.append({'audio': [
  'list_audio(path, midi=False)',
  'swf(sig1, sig2=\'\', peaks=[], rnd=False, sr=44100)',
  'waveform(input, dur=None, peaks=[], sr=44100)',
  'pitchform(wav_file)',
  'waveform_pitch(wav_file)',
  'audio_player(input, sr=44100, limit_duration=2)',
  'waveform_player(mp3, wav)',
  'configSpleeter()',
  ''
]})

import soundfile as sf
from scipy.signal import butter, lfilter, freqz

# Input (str): directory path
# Output (list): list of audiofiles in dir
def list_audio(path, midi=False):
  audiofiles = []
  for ext in ('*.wav', '*.aiff', '*.aif', '*.caf' '*.flac', '*.mp3', '*.m4a', '*.ogg', '*.WAV', '*.AIFF', '*.AIF', '*.CAF', '*.FLAC', '*.MP3', '*.OGG'):
    audiofiles.extend(glob(join(path, ext)))
  if midi == True:
    for ext in ('*.mid', '*.midi', '*.MID', '*.MIDI'):
      audiofiles.extend(glob(join(path, ext)))
  audiofiles.sort()
  return audiofiles

def swf(sig1, sig2='', peaks=[], rnd=False, sr=44100):
  #yellowgreen, salmon
  duration = len(sig1)/sr
  time = np.arange(0,duration,1/sr)
  plt.rcParams.update({"axes.facecolor": "black"})
  plt.ylim(-1, 1)
  if rnd==True:
    c = np.random.rand(3)
  else:
    c = '#00ffdd'
  plt.axhline(y=0, color='#fff', linewidth=0.5, alpha=0.5)
  plt.plot(time, sig1, color=c, linewidth=0.3, alpha=1)
  if len(peaks) > 0:
    for i, peak_set in enumerate(peaks):
      #print(i, peak_set)
      if i == 0:
        c = '#f3d'
        prio = .65
        lw = 0.7
      else:
        c = '#fff'
        prio = .6
        lw = 0.7
      for peak in peak_set:
        plt.axvline(x=peak/sr, color=c, linewidth=lw, alpha=prio)
  if sig2 != '':
    plt.plot(time, sig2, color=np.random.rand(3), linewidth=0.3, alpha=0.55)
  plt.show()

# Input (any): wav file path or audio signal as ndarray, duration, peak, samplerate
# Output: waveform image of audio signal
def waveform(input, dur=None, peaks=[], sr=44100):
  if type(input) == np.ndarray:
    data = input
  else:
    data, sr = librosa.load(input, sr=sr, duration=dur, offset=0.0)
  plt.rcParams['axes.facecolor'] = plot_bg
  fig = plt.figure(figsize=(16, 5), frameon=False)
  #ax = fig.add_axes([0, 0, 1, 1])
  #ax.axis('off')
  plt.axis([-1, 1, -1, 1])
  #plt.ylim(-1, 1)
  if len(peaks) > 0:
    for peak in peaks:
      plt.axvline(x=peak, color='r')
  librosa.display.waveplot(data, sr=sr, color=plot_wav)
  plt.show()

# Input (str): path to WAV file
# Output: image of pitches
def pitchform(wav_file):
  y, sr = librosa.load(wav_file)
  pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
  plt.figure(figsize=(16, 8))
  plt.imshow(pitches[:100, :], aspect="auto", interpolation="nearest", origin="bottom")
  plt.show()

# Input (str): path to WAV file
# Output: image of waveform and pitches
def waveform_pitch(wav_file):
  waveform(wav_file)
  pitchform(wav_file)

def audio_player(input, sr=44100, limit_duration=2, mono=False):
  if type(input) != np.ndarray:
    input, sr = librosa.load(input, sr=None, mono=mono)
  if limit_duration > 0:
    last_sample = math.floor(limit_duration*60*sr)
    if input.shape[-1] > last_sample:
      input = input[:last_sample, :last_sample]
      op(c.warn, 'WARN! Playback of below audio player is limited to first '+str(limit_duration)+' minutes to prevent Colab from crashing.\n')
  IPython.display.display(IPython.display.Audio(input, rate=sr))

# Input (str): path to MP3 file, path to WAV file (of the same track)
# Output: image of waveform, image of pitches and audio player
def waveform_player(mp3, wav):
  waveform_pitch(wav)
  audio_player(mp3)

# Output: creates current /content/cfg.json
def configSpleeter():
  !gsutil -q -m cp -R gs://neural-research/olaviinha/spleeter-configs/custom-4stems-22kHz-a.json /content/cfg.json

# --- Librosa utils ----

# Clip audio signal
# Returns clipped audio siangl
def clip_audio(audio_data, sr=44100, start=0, duration=10, fx=[], oneshots=False):
  xstart = librosa.time_to_samples(start, sr=sr)
  xduration = librosa.time_to_samples(start+duration, sr=sr)
  if audio_data.ndim > 1:
    audio_data = audio_data[:, xstart:xduration]
  else:
    audio_data = audio_data[xstart:xduration]
  return audio_data

# Apply fade in and/or fade out to audio signal
# Returns faded audio signal
def fade_audio(audio_data, fade_in=0, fade_out=0, sr=44100):
  a_duration = librosa.get_duration(y=audio_data, sr=sr)
  if fade_in > 0:
    fade_in_to = librosa.time_to_samples(fade_in, sr=sr)
    in_y = audio_data[:, 0:fade_in_to]
    fade_ins = []
    for channel in in_y:
      fade = [ i/len(channel)*smp for i, smp in enumerate(channel) ]
      fade_ins.append(fade)
    fade_ins = np.array(fade_ins)
    tail_start = fade_in_to+1
    tail = audio_data[:, tail_start:]
    audio_data = np.concatenate([fade_ins, tail], axis=1)
  if fade_out > 0:
    fade_out_start = librosa.time_to_samples(a_duration-fade_out, sr=sr)
    out_y = audio_data[:, fade_out_start:]
    fade_outs = []
    for channel in out_y:
      fade = [ smp-(i/len(channel)*smp) for i, smp in enumerate(channel) ]
      fade_outs.append(fade)
    fade_outs = np.array(fade_outs)
    head_start = fade_out_start-1
    head = audio_data[:, :head_start]
    audio_data = np.concatenate([head, fade_outs], axis=1)
  return audio_data

def narrow_stereo(left_data, right_data, amount):
  left = left_data * amount + right_data * (1-amount)
  right = right_data * amount + left_data * (1-amount)
  return np.array([left, right])

def highpass_audio(input, cutoff=10000, fs=44100, order=6):
  if type(input) == np.ndarray:
    data = input
    sr = fs
  else:
    data, sr = librosa.load(input, sr=None, mono=False)
  b, a = butter(order, cutoff, fs=fs, btype='highpass', analog=False)
  y = lfilter(b, a, data)
  return y

def apply_highpass_mix(input, hipass_vol=0.5, mix_vol=0.8, cutoff=10000, fs=44100, order=6):
  if type(input) == np.ndarray:
    data = input
    sr = fs
  else:
    data, sr = librosa.load(input, sr=None, mono=False)
  octave_up = librosa.effects.pitch_shift(data, sr=sr, n_steps=12) * hipass_vol
  hipass = highpass_audio(octave_up, cutoff, fs, order)
  return data*mix_vol+hipass

def split_channels(audio_data):
  return audio_data[0], audio_data[1]

def merge_channels(left_data, right_data):
  return np.array([left_data, right_data])

def time_stretch_audio(audio, sr, to_length):
  dur = librosa.get_duration(y=audio, sr=sr)
  if audio.ndim > 1:
    return np.array([librosa.effects.time_stretch(channel, dur/to_length) for channel in split_channels(audio)])
  return np.array(librosa.effects.time_stretch(audio, dur/to_length))

def repeat_or_clip_audio(audio, sr, to_length):
  dur = librosa.get_duration(y=audio, sr=sr)
  from_samples = librosa.time_to_samples(dur, sr=sr)
  to_samples = librosa.time_to_samples(to_length, sr=sr)
  is_stereo = True if audio.ndim > 1 else False
  if from_samples < to_samples:
    repeats = math.floor(to_samples/from_samples)
    leftover = to_samples-from_samples
    new_audio = np.tile(audio, repeats)
    leftover_audio = audio[:leftover, :leftover] if is_stereo else audio[:leftover]
    new_audio = np.concatenate((new_audio, leftover_audio), axis=1 if is_stereo else 0)
  else:
    new_audio = audio[:to_samples, :to_samples] if is_stereo else audio[:to_samples]
  return new_audio

def remove_silence(audio, window_size=0.2, threshold=0.1, save_as='', sr=44100):
  if type(audio) != np.ndarray:
    y, sr = librosa.load(audio, sr=None, mono=False)
  else:
    y = audio
  audio_slices = ster_slice_to_frames(y, window_size, sr=sr)
  silence_removed_list = []
  for audio_slice in audio_slices:
    if max(audio_slice[0]) > threshold or max(audio_slice[1]) < -abs(threshold):
      silence_removed_list.append(audio_slice)
  silence_removed = np.concatenate(silence_removed_list, axis=1)
  if save_as != '':
    sf.write(save_as, silence_removed.T, sr)
    return save_as
  return silence_removed

def slice_to_frames(audio_data, sr=44100, slice_duration=1, fade_in=0, fade_out=0, fx=[]):
  a_duration = librosa.get_duration(y=audio_data, sr=sr)
  clips = math.ceil(a_duration/slice_duration)
  frames = []
  for i in range(clips):
    # if i > 0 and i < clips:
    # if i > 0:
    start = i*slice_duration
    audio_clip = clip_audio(audio_data, sr, start, slice_duration, fx)
    frames.append( audio_clip ) #fade_audio(audio_clip, fade_in, fade_out) )
  return frames

# Clip audio signal
# Returns clipped audio siangl
def clip_audio(audio_data, sr=44100, start=0, duration=10, fx=[], oneshots=False):
  xstart = librosa.time_to_samples(start, sr=sr)
  xduration = librosa.time_to_samples(start+duration, sr=sr)
  if audio_data.ndim > 1:
    audio_data = audio_data[:, xstart:xduration]
  else:
    audio_data = audio_data[xstart:xduration]
  return audio_data

# Apply fade in and/or fade out to audio signal
# Returns faded audio signal
def fade_audio(audio_data, fade_in=0, fade_out=0, sr=44100):
  a_duration = librosa.get_duration(y=audio_data, sr=sr)
  if fade_in > 0:
    fade_in_to = librosa.time_to_samples(fade_in, sr=sr)
    in_y = audio_data[:, 0:fade_in_to]
    fade_ins = []
    for channel in in_y:
      fade = [ i/len(channel)*smp for i, smp in enumerate(channel) ]
      fade_ins.append(fade)
    fade_ins = np.array(fade_ins)
    tail_start = fade_in_to+1
    tail = audio_data[:, tail_start:]
    audio_data = np.concatenate([fade_ins, tail], axis=1)
  if fade_out > 0:
    fade_out_start = librosa.time_to_samples(a_duration-fade_out, sr=sr)
    out_y = audio_data[:, fade_out_start:]
    fade_outs = []
    for channel in out_y:
      fade = [ smp-(i/len(channel)*smp) for i, smp in enumerate(channel) ]
      fade_outs.append(fade)
    fade_outs = np.array(fade_outs)
    head_start = fade_out_start-1
    head = audio_data[:, :head_start]
    audio_data = np.concatenate([head, fade_outs], axis=1)
  return audio_data

def ster_slice_to_frames(audio_data, slice_duration, fade_in=0, fade_out=0, sr=44100):
  a_duration = librosa.get_duration(y=audio_data, sr=sr)
  clips = math.ceil(a_duration/slice_duration)
  frames = []
  for i in range(clips-1):
    if i > 0 and i < clips:
      start = i*slice_duration
      audio_clip = ster_clip_audio(audio_data, start, slice_duration)
      if fade_in > 0 or fade_out > 0:
        audio_clip = ster_fade_audio(audio_clip, fade_in, fade_out, sr=sr)
      frames.append(audio_clip)
  return frames

def ster_clip_audio(audio_data, start, duration, sr=44100):
  xstart = librosa.time_to_samples(start, sr=sr)
  xduration = librosa.time_to_samples(start+duration, sr=sr)
  if audio_data.ndim == 2:
    audio_data = audio_data[:, xstart:xduration]
  else:
    audio_data = audio_data[xstart:xduration]
  return audio_data

def ster_fade_audio(audio_data, fade_in=0.05, fade_out=0.05, sr=44100):
  a_duration = librosa.get_duration(y=audio_data, sr=sr)
  if fade_in > 0:
    fade_in_to = librosa.time_to_samples(fade_in, sr=sr)
    if audio_data.ndim == 2:
      in_y = audio_data[:, 0:fade_in_to]
    else:
      in_y = audio_data[0:fade_in_to]
    fade_ins = []
    for channel in in_y:
      fade = [ i/len(channel)*smp for i, smp in enumerate(channel) ]
      fade_ins.append(fade)
    fade_ins = np.array(fade_ins)
    tail_start = fade_in_to+1
    if audio_data.ndim == 2:
      tail = audio_data[:, tail_start:]
    else:
      tail = audio_data[tail_start:]
    # audio_data = np.concatenate([fade_ins, tail], axis=1)
    audio_data = np.concatenate([fade_ins, tail], axis=audio_data.ndim-1)
  if fade_out > 0:
    fade_out_start = librosa.time_to_samples(a_duration-fade_out, sr=sr)
    if audio_data.ndim == 2:
      out_y = audio_data[:, fade_out_start:]
    else:
      out_y = audio_data[fade_out_start:]
    fade_outs = []
    if audio_data.ndim == 2:
      for channel in out_y:
        fade = [ smp-(i/len(channel)*smp) for i, smp in enumerate(channel) ]
        fade_outs.append(fade)
    else:
      fade = smp-(i/len(out_y)*smp)
      fade_outs.append(fade)
    fade_outs = np.array(fade_outs)
    head_start = fade_out_start-1
    if audio_data.ndim == 2:
      head = audio_data[:, :head_start]
    else:
      head = audio_data[:head_start]
    # audio_data = np.concatenate([head, fade_outs], axis=1)
    audio_data = np.concatenate([head, fade_outs], axis=audio_data.ndim-1)
  return audio_data

### Test

In [ ]:
# create_dirs(['/a', '/a/b', '/a/b/c'])
# !cp /content/sample_data/anscombe.json /a/b/c.exs
# existing_file = '/a/b/c.exs'
# nonexisting_file = '/a/b/c.non'
# existing_path1 = '/a/b/c'
# existing_path2 = '/a/b/c/'
# testlist = ['a', 'a', 'b', 'c', 'd', 'd', 'd']
# crazyfile = "!This is a% (Real)[Crazy] File-name~1.MKV"

# print('Notebook is running in hosted Colab env:',   is_hosted_runtime() )
# print('Add end slash', existing_path1, '->',        fix_path(existing_path1))
# print('Remove start slash', existing_path2, '->',   fix_path(existing_path2, True))
# print('Last item from file path', existing_file, '->', path_leaf(existing_file))
# print('Last item from dir path', existing_path1, '->', path_leaf(existing_path1))
# print('Dir path from path', existing_file, '->', path_dir(existing_file))
# print('Basename from path', existing_file, '->', basename(existing_file))
# print('.Extension from path', existing_file, '->', path_ext(existing_file))
# print('Extension from path', existing_file, '->', path_ext(existing_file, True))
# print('Check input type for existing file path', existing_file, '->', check_input_type(existing_file))
# print('Check input type for existing dir path', existing_path1, '->', check_input_type(existing_path1))
# print('Check input type for nonexisting file', nonexisting_file, '->', check_input_type(nonexisting_file))
# print('Check most frequent value in list', testlist, '->', most_frequent(testlist))
# print('Prettify filename', crazyfile, '->', slug(crazyfile))
# print('Random string of 8 characters ->', rnd_str(8))
# print('Concatenate list with separator "-v 0.3" ->', concat_list("-v 0.3", testlist))
# print('Return True with 20% probability ->', odds(0.2))
# print('Return True with 50% probability ->', odds(0.5))
# print('Return True with 80% probability ->', odds(0.8))